In [1]:
#가능한 테스트 세트는 마지막 1회만 쓰는게 좋음.->검증세트를 추가하여 테스트.
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

sub_input, val_input, sub_target, val_target = train_test_split(#검증 세트를 위한 분할
    train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [2]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [3]:
#검증 세트를 만든만큼 훈련세트가 줄어듬.->교차 검증을 통해 훈련 세트를 더 많이 확보하는 효과를 확보할 수 있다.
#검증세트와 훈련세트를 교차로 섞어 훈련세트로 훈련하고 검증세트로 평가 : k-fold 교차검증
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)# 기본 5겹교차검증.(cv=5)
print(scores)#test_score가 검증한것.(테스트세트쓴것 아니다.)
#딥러닝에서는 교차검증을 하지 않음.(데이터가 많기때문)
#머신러닝에서는 데이터가 많지않아 교차검증 사용

{'fit_time': array([0.00531483, 0.00501513, 0.00553799, 0.00518513, 0.00479889]), 'score_time': array([0.00062704, 0.00053883, 0.00062585, 0.00063992, 0.00055623]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [4]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [5]:
#cross_validate()함수에서는 분할 전 훈련세트를 섞지 않는다. 섞기위해서 사이킷런의 분할기 사용
from sklearn.model_selection import StratifiedKFold #분류 모델의 분할기(회귀 모델의 분할기는 KFold 이다.)

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [6]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)   #별도의 객체(splitter)를 사용해 훈련세트를 섞고(shuffle) 10폴드(n_splits) 교차검증
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 그리드 서치

In [7]:
# 머신러닝이 학습하는 파라미터 : 모델 파라미터(가중치, 편향, 결정 트리의 질문 등)
# 모델이 학습 할 수 없어서(선택사항으로 설계자가 결정) 사용자가 지정해야 하는 파라미터 : 하이퍼 파라미터
#하이퍼 파라미터는 클래스나 함수의 매개변수로 표현.

#그리드 서치 : 최적의 매개변수를 찾고 매개변수가 n개일 경우 이들의 최적의 조합까지 자동으로 찾아주는 방법

In [8]:
from sklearn.model_selection import GridSearchCV #cross_validate()함수 호출 필요 없다.(교차검증, 매개변수 변경 자동화)

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} #최소 불순 감소도 측정 매개변수. 정보이득 최솟값 지정

In [9]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) #params 지정, 모든 코어 사용
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [10]:
dt = gs.best_estimator_ #가장 높은 매개변수의 조합 세트
print(dt.score(train_input, train_target))

0.9615162593804117


In [12]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [14]:
print(gs.cv_results_['mean_test_score'])  #5개 매개변수 각각의 5겹 교차 검증 수행 결과

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [15]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [16]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          } #9*15*10 * 5겹 = 6750

In [17]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [18]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [19]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# 확률 분포 선택
## 랜덤서치

In [20]:
from scipy.stats import uniform, randint
# 실수, 정수용 균든 분포 샘플링
rgen = randint(0, 10)
rgen.rvs(10) #rvs() 표본값 무작위 생성함수

array([5, 0, 3, 8, 1, 8, 3, 5, 2, 4])

In [21]:
np.unique(rgen.rvs(1000), return_counts=True)# unique값 출력, 갯수 출력

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([101, 108,  94, 112, 102, 100,  93,  99,  86, 105]))

In [22]:
ugen = uniform(0, 1) #실수일 경우 사용, 난수 발생기와 유사
ugen.rvs(10)

array([0.05498382, 0.35435532, 0.23922521, 0.39927687, 0.7048454 ,
       0.19272048, 0.90525392, 0.29855926, 0.35149298, 0.0478201 ])

In [23]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25), #리프노드가 되기 위한 최소한의 샘플
          }

In [24]:
from sklearn.model_selection import RandomizedSearchCV #params 범위 내에서 랜덤 샘플링 횟수 지정을 위한 클래스

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fee09da6250>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fee09e42b50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fee09da6150>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fee09da6590>},
                   random_state=42)

In [25]:
print(gs.best_params_) #최적의 매개변수 조합
print(np.max(gs.cv_results_['mean_test_score'])) #최적의 매개변수 조합의 교차검증 점수

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [26]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))#최적의 모델을 dt로 받아 테스트 세트로 수행

0.86


In [27]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fee09da6250>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fee09e42b50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fee09da6150>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fee09da6590>},
                   random_state=42)

In [28]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077


검증 세트  
모델을 평가할 때 테스트 세트를 사용하지 않기 위해 훈련세트에서 다시 떼어내 데이터 세트  
하이퍼파라미터 튜닝 등의 작업에 효과적  
  
교차검증  
훈련세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트의 역할을 하고 나머지 폴드에서는 모들을 훈련  
모든 폴드에 대해 검증 점수를 얻어 평균하는 방법. 3-폴드 교차 검증 : 훈련 세트를 세 부분으로 나눠서 교차 검증을 수행하는 것  
  
그리드 서치  
하이퍼파라미터 탐색을 자동화해 주는 도구  
탐색할 매개변수를 나열해 놓으면 교차 검증을 수행하여 가장좋은 검증 점수의 매개변수 조합을 선택, 마지막으로 이 매개변수 조합으로 최종 모델을 훈련  
  
랜덤서치  
연속된 매개변수 값을 탐색할 때 유용. 탐색할 값을 직접 나열하는 것이 아니라, 탐색 값을 샘플링 할 수 있는 확률 분포 객체를 전함  
지정된 횟수만큼 샘플링하여 교차 검증을 수행하기 때문에 시스템 자원이 허락하는 만큼 탐색량을 조절할 수 있음